## The purprose of this notebook is to research on [huggingface sql embedding mode](https://huggingface.co/s2593817/sft-sql-embedding)  
* The main objective is to assess how well this model can provide an accurate cosine similarity score between two sql queries.
* We will compare this with continuous_eval sql metrics score and SequenceMatcher from difflib 

In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np
from difflib import SequenceMatcher
from continuous_eval.metrics.code.sql import SQLASTSimilarity

In [15]:
embedding_model = SentenceTransformer("s2593817/sft-sql-embedding")
SQLAST_METRIC = SQLASTSimilarity()

In [19]:
def calculate_cosine_similarity(query_a:str, query_b:str) -> float:
    embeddings = embedding_model.encode([query_a, query_b])
    cosine_similarity_score = embedding_model.similarity(embeddings, embeddings)
    return cosine_similarity_score[0][1]

def calculate_sequence_matcher_similarity(query_a:str, query_b:str) -> float:
    return SequenceMatcher(None, query_a, query_b).ratio()

def calculate_sql_ast_similarity(query_a: str, query_b: str) -> float:
    return SQLAST_METRIC.compute(answer=query_a, ground_truth_answers=query_b)["SQL_AST_Similarity"]

def compare_scores(query_a: str, query_b: str) -> None:
    cosine_similarity_score = calculate_cosine_similarity(query_a, query_b)
    sequence_matcher_score = calculate_sequence_matcher_similarity(query_a, query_b)
    sql_ast_metric = calculate_sql_ast_similarity(query_a, query_b)
    print(f"cosine similarity score: {cosine_similarity_score}\n")
    print(f"sequence matcher similarity score: {sequence_matcher_score}\n")
    print(f"SQL AST metric score: {sql_ast_metric}\n")

## Compare when two queries are different (Negative Cases)

## completely different queries

In [24]:
query_a = """
SELECT
    age,
    hire_date,
    department,
    tenure
FROM
    employee_table
QUALIFY max(salary) over (partition by hire_date) = salary
"""

query_b = """
SELECT
    max(salary)
FROM 
    employee_table
GROUP BY age
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.2638276219367981

sequence matcher similarity score: 0.46078431372549017

SQL AST metric score: 0.43181818181818177



In [25]:
query_a = """
SELECT
    DISTINCT ACCOUNT_ID
FROM 
    transactions
WHERE
    TRANSACTION_DATE > '2025-07-01'
"""

query_b = """
SELECT
    * 
FROM 
    accounting
WHERE
    application IS NOT NULL
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.40218862891197205

sequence matcher similarity score: 0.5149700598802395

SQL AST metric score: -0.5



### check for Subtle differences in filtering and column name differences

In [49]:
query_a = """
SELECT
    DISTINCT ACCOUNT_ID
FROM 
    transactions
WHERE
    TRANSACTION_DATE > '2025-07-01'
"""

query_b = """
SELECT
    DISTINCT ACCOUNT_ID
FROM 
    transactions
WHERE
    TRANSACTION_DATE < '2025-07-01'
"""

query_c = """
SELECT
    DISTINCT ACCOUNT_ID
FROM 
    transactions
WHERE
    TRANSACTION_DATE <> '2025-07-01'
"""
compare_scores(query_a, query_b)
compare_scores(query_a, query_c)
compare_scores(query_b, query_c)

cosine similarity score: 0.9975489974021912

sequence matcher similarity score: 0.9896907216494846

SQL AST metric score: 0.75

cosine similarity score: 0.9954880475997925

sequence matcher similarity score: 0.9948717948717949

SQL AST metric score: 0.75

cosine similarity score: 0.9972676634788513

sequence matcher similarity score: 0.9948717948717949

SQL AST metric score: 0.75



In [51]:
query_a = """
SELECT
    department
FROM
    employee_table
QUALIFY max(salary) over (partition by hire_date) = salary
"""
query_b = """
SELECT
    employee_name 
FROM
    employee_table
QUALIFY max(hire_date) over (partition by department) = hire_date
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.9711019992828369

sequence matcher similarity score: 0.7713004484304933

SQL AST metric score: 0.1875



### Check for Similar Queries

### seq changes 

In [31]:
query_a = """
SELECT 
    log_date,
    device_name,
    department,
    application_name,
    device_id
FROM
    device_tracking
ORDER BY log_date, device_id
"""
query_b = """
SELECT 
    device_id,
    device_name,
    application_name,
    department,
    log_date
FROM
    device_tracking
ORDER BY device_id, log_date
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.995932400226593

sequence matcher similarity score: 0.7534246575342466

SQL AST metric score: 0.9047619047619048



### Aliases

In [36]:
query_a = """
SELECT 
    tbl.log_date as "log date",
    tbl.device_name as "device name",
    tbl.department,
    tbl.application_name,
    tbl.device_id as "device id"
FROM
    device_tracking as tbl
GROUP BY 1,2,3,4,5
"""
query_b = """
SELECT 
    device_id,
    device_name,
    application_name,
    department,
    log_date
FROM
    device_tracking
GROUP BY device_id, device_name, application_name, department, log_date
"""
query_c = """
SELECT 
    device_id,
    device_name,
    application_name,
    department,
    log_date
FROM
    device_tracking
GROUP BY ALL
"""
compare_scores(query_a, query_b)
compare_scores(query_a, query_c)
compare_scores(query_b, query_c)

cosine similarity score: 0.8188620209693909

sequence matcher similarity score: 0.5376884422110553

SQL AST metric score: 0.10606060606060608

cosine similarity score: 0.5447807312011719

sequence matcher similarity score: 0.6076696165191741

SQL AST metric score: 0.25757575757575757

cosine similarity score: 0.898712694644928

sequence matcher similarity score: 0.7962382445141066

SQL AST metric score: 0.52



### check for sql dialects: date function and subquery/cte/qualify

In [38]:
query_a = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time >= '2025-07-11' and last_login_time <= '2025-07-15'
"""

query_b = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time BETWEEN '2025-07-11' and '2025-07-15'
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.9567475914955139

sequence matcher similarity score: 0.910828025477707

SQL AST metric score: 0.675



In [41]:
query_a = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time >= TRUNC(SYSDATE) - 7
"""

query_b = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time >= DATE_SUB(CURDATE(), INTERVAL 7 DAY)
"""

query_c = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time >= CURRENT_DATE - INTERVAL '7' DAY
"""

query_d = """
SELECT
    customer_name,
    create_date,
    last_login_time
FROM 
    customers 
WHERE 
    last_login_time BETWEEN (CURRENT_DATE - INTERVAL '7' DAY) AND NOW();
"""
compare_scores(query_a, query_b)
compare_scores(query_a, query_c)
compare_scores(query_a, query_d)
compare_scores(query_b, query_c)
compare_scores(query_b, query_d)
compare_scores(query_c, query_d)

cosine similarity score: 0.8955351710319519

sequence matcher similarity score: 0.8701754385964913

SQL AST metric score: 0.2894736842105263

cosine similarity score: 0.9147263169288635

sequence matcher similarity score: 0.8896797153024911

SQL AST metric score: 0.2894736842105263

cosine similarity score: 0.8297773599624634

sequence matcher similarity score: 0.822742474916388

SQL AST metric score: 0.0714285714285714

cosine similarity score: 0.9436810612678528

sequence matcher similarity score: 0.9060402684563759

SQL AST metric score: 0.7631578947368421

cosine similarity score: 0.9057097434997559

sequence matcher similarity score: 0.8481012658227848

SQL AST metric score: 0.5714285714285714

cosine similarity score: 0.9116183519363403

sequence matcher similarity score: 0.9294871794871795

SQL AST metric score: 0.7619047619047619



In [42]:
query_a = """
SELECT
    customer_name,
    YEAR(create_date) as year,
    DAY(last_login_time) as DAY
FROM 
    customers 
"""

query_b = """
SELECT
    customer_name,
    EXTRACT(YEAR FROM create_date) as year,
    EXTRACT(DAY FROM last_login_time) as DAY
FROM 
    customers 
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.9536982178688049

sequence matcher similarity score: 0.8790322580645161

SQL AST metric score: 0.4444444444444444



### regex and string function

In [47]:
query_a = """
SELECT
    LEFT(DeviceName, 7),
    RIGHT(DeviceName, 3),
    IpSource || '-' || IpTarget,
    COUNT(*)
FROM servers
WHERE DeviceName Like '%HOME%' and IpSource != '172.16.14.16'
"""
query_b = """
SELECT
    SUBSTRING(DeviceName, 1, 7),
    SUBSTRING(DeviceName, LEN(DeviceName)-2, LEN(DeviceName)),
    CONCAT(IpSource, '-', IpTarget),
    COUNT(1),
FROM servers
WHERE DeviceName REGEXP '%HOME%' and IpSource <> '172.16.14.16'
"""
compare_scores(query_a, query_b)

cosine similarity score: 0.8834095597267151

sequence matcher similarity score: 0.5679611650485437

SQL AST metric score: 0.3421052631578947



### Qualify versus Subquery versus CTE

In [48]:
query_a = """
SELECT
    USER_ID,
    SIGN_UP_DATE,
    STATE,
    PURCHASE
FROM 
    CUSTOMER_PURCHASES
QUALIFY MAX(SIGN_UP_DATE) OVER (PARTITION BY STATE) = SIGN_UP_DATE
"""

query_b = """
WITH stateData As (
    SELECT
        USER_ID,
        SIGN_UP_DATE,
        STATE,
        PURCHASE,
        ROW_NUMBER() OVER (PARTITION BY STATE) as row_number
    FROM
        CUSTOMER_PURCHASES        
)
SELECT
    USER_ID,
    SIGN_UP_DATE,
    STATE,
    PURCHASE
FROM stateData
WHERE row_number = 1
"""

query_c = """
SELECT
    USER_ID,
    SIGN_UP_DATE,
    STATE,
    PURCHASE
FROM (
    SELECT
        USER_ID,
        SIGN_UP_DATE,
        STATE,
        PURCHASE,
        ROW_NUMBER() OVER (PARTITION BY STATE) as row_number
    FROM
        CUSTOMER_PURCHASES
) 
WHERE row_number = 1
"""
compare_scores(query_a, query_b)
compare_scores(query_a, query_c)
compare_scores(query_b, query_c)

cosine similarity score: 0.7591825127601624

sequence matcher similarity score: 0.3247863247863248

SQL AST metric score: 0.34615384615384615

cosine similarity score: 0.8407358527183533

sequence matcher similarity score: 0.4618937644341801

SQL AST metric score: 0.3382352941176471

cosine similarity score: 0.9755663871765137

sequence matcher similarity score: 0.7101200686106347

SQL AST metric score: 0.8076923076923077

